### MaskRCNN training. Common objects in context COCO dataset

In [1]:
import os
os.chdir('..')
import tensorflow as tf

from samples.coco import coco
from preprocess import preprocess
from preprocess import augmentation as aug
from training import train_model
from model import mask_rcnn_functional
from common.utils import tf_limit_gpu_memory

In [2]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2021-06-24T23:47:13.332198+03:00

Python implementation: CPython
Python version       : 3.7.7
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-65-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

tensorflow: 2.2.0



In [3]:
tf_limit_gpu_memory(tf, 4500)

1 Physical GPUs, 1 Logical GPUs Memory limit: 4500
Physical GPU-devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from common.config import CONFIG

In [5]:
CONFIG.update(coco.COCO_CONFIG)
CONFIG

{'image_shape': (512, 512, 3),
 'img_size': 512,
 'backbone': 'mobilenet',
 'meta_shape': 93,
 'num_classes': 81,
 'class_dict': {'background': 0,
  'person': 1,
  'bicycle': 2,
  'car': 3,
  'motorcycle': 4,
  'airplane': 5,
  'bus': 6,
  'train': 7,
  'truck': 8,
  'boat': 9,
  'traffic light': 10,
  'fire hydrant': 11,
  'stop sign': 12,
  'parking meter': 13,
  'bench': 14,
  'bird': 15,
  'cat': 16,
  'dog': 17,
  'horse': 18,
  'sheep': 19,
  'cow': 20,
  'elephant': 21,
  'bear': 22,
  'zebra': 23,
  'giraffe': 24,
  'backpack': 25,
  'umbrella': 26,
  'handbag': 27,
  'tie': 28,
  'suitcase': 29,
  'frisbee': 30,
  'skis': 31,
  'snowboard': 32,
  'sports ball': 33,
  'kite': 34,
  'baseball bat': 35,
  'baseball glove': 36,
  'skateboard': 37,
  'surfboard': 38,
  'tennis racket': 39,
  'bottle': 40,
  'wine glass': 41,
  'cup': 42,
  'fork': 43,
  'knife': 44,
  'spoon': 45,
  'bowl': 46,
  'banana': 47,
  'apple': 48,
  'sandwich': 49,
  'orange': 50,
  'broccoli': 51,
  'ca

In [6]:
model = mask_rcnn_functional(config=CONFIG)

[MaskRCNN] Training mode

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[MaskRCNN] Backbone architecture: mobilenet


/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [7]:
CONFIG['training']

True

In [8]:
CONFIG['backbone']

'mobilenet'

In [9]:
base_dir = r'/media/alexander/Samsung_T5/temp/coco2017' 
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

train_dataset = coco.CocoDataset(dataset_dir=train_dir,
                                 subset='train',
                                 year=2017,
                                 auto_download=False,
                               
                                 # SegmentationDataset necessary parent attributes
                                 augmentation=aug.get_training_augmentation(
                                               image_size=CONFIG['img_size'],
                                               normalize=CONFIG['normalization']
                                 ),
                                 **CONFIG
                                )

val_dataset = coco.CocoDataset(dataset_dir=val_dir,
                               subset='val',
                               year=2017,
                               auto_download=False,
                               
                               # SegmentationDataset necessary parent attributes
                               augmentation=aug.get_validation_augmentation(
                                           image_size=CONFIG['img_size'],
                                           normalize=CONFIG['normalization']
                               ),
                               **CONFIG
                              )

train_dataloader = preprocess.DataLoader(train_dataset,
                                         shuffle=True,
                                         name='train',
                                         **CONFIG
                                        )
val_dataloader = preprocess.DataLoader(val_dataset,
                                       shuffle=False,
                                       name='val',
                                       **CONFIG
                                      )

None passed to images_dir argument.
 This means that the dataset class is a child of SegmentationDataset and its behaviour differs from datasets created with VGG Image Annotator.
 If it is not true, please, check your class arguments carefully.

loading annotations into memory...
Done (t=9.88s)
creating index...


  0%|          | 0/117266 [00:00<?, ?it/s]

index created!


100%|██████████| 117266/117266 [00:01<00:00, 81825.15it/s]


None passed to images_dir argument.
 This means that the dataset class is a child of SegmentationDataset and its behaviour differs from datasets created with VGG Image Annotator.
 If it is not true, please, check your class arguments carefully.

loading annotations into memory...


100%|██████████| 4952/4952 [00:00<00:00, 84707.84it/s]

Done (t=2.31s)
creating index...
index created!
train DataLoader. Steps per epoch: 117266
val DataLoader. Steps per epoch: 4952


In [ ]:
train_model(model, 
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            config=CONFIG, 
            weights_path=None)

train DataLoader. Steps per epoch: 117266
val DataLoader. Steps per epoch: 4952
MaskRCNN Losses:
rpn_class_loss: <layers.losses.RPNClassLoss object at 0x7ff0c0746550>
rpn_bbox_loss: <layers.losses.RPNBboxLoss object at 0x7feebc6adb10>
mrcnn_class_loss: <layers.losses.MRCNNClassLoss object at 0x7feebc6adad0>
mrcnn_bbox_loss: <layers.losses.MRCNNBboxLoss object at 0x7ff0c81d2e90>
mrcnn_mask_loss: <layers.losses.MRCNNMaskLoss object at 0x7ff0c84a5050>
l2_regularizer: <layers.losses.L2RegLoss object at 0x7feebc6ad990>

Epoch 1/100
Instructions for updating:
Use tf.identity instead.
     5/117266 [..............................] - ETA: 41:01:24 - rpn_class_loss: 6.4406 - rpn_bbox_loss: 3.9042 - mrcnn_class_loss: 1.7997 - mrcnn_bbox_loss: 7.5495 - mrcnn_mask_loss: 0.9319 - l2_regularizer: 0.0084 - loss_sum: 20.6343

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-080c3c381784>", line 5, in <module>
    weights_path=None)
  File "/home/alexander/Documents/py_projects/bitbucket/mask-rcnn/src/training.py", line 138, in train_model
    workers=config['workers'],
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 848, in fit
    tmp_logs = train_function(iterator)
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 572, in train_function
    self.train_step, args=(data,))
  File "

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-080c3c381784>", line 5, in <module>
    weights_path=None)
  File "/home/alexander/Documents/py_projects/bitbucket/mask-rcnn/src/training.py", line 138, in train_model
    workers=config['workers'],
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 66, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 848, in fit
    tmp_logs = train_function(iterator)
  File "/home/alexander/anaconda3/envs/tf_env/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 572, in train_function
    self.train_step, args=(data,))
  File "